# OTX API DEMO (Anomalib Example)

## Customization Training API

Select a framework & import adapter modules.

We'll choose Anomalib here, and we'll import the following modules.

In [1]:
from otx.v2.adapters.torch.anomalib import Dataset, get_model, Engine

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


To use wandb logger install it using `pip install wandb`


## Prepare Dataset & DataLoader
1. Prepare a dataset and enter path into Dataset

    - Convert to OTX's DatasetEntity and Label Schema by leveraging Datumaro's features through paths (path -> Datumaro -> OTX DatasetEntity & LabelSchema)

In [2]:
dataset = Dataset(
    train_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/anomaly/hazelnut/train",
    val_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/anomaly/hazelnut/test",
    test_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/anomaly/hazelnut/test",
)

In [3]:
train_dataloader = dataset.train_dataloader()
print(f"Dataset type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")

[*] Detected task type: ANOMALY_CLASSIFICATION
Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 28
Dataset size: 28


In [4]:
val_dataloader = dataset.val_dataloader()
print(f"Dataset type: {type(val_dataloader)}")
print(f"Length of DataLoader: {len(val_dataloader)}")
print(f"Dataset size: {len(val_dataloader.dataset)}")

Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 23
Dataset size: 23


## Prepare Model
Config to build the model. -> Provide function for building models so that each framework's config can be used

    - Users can build a torch.nn.Module via config as well

In [5]:
model_config = {
    "model": {
        "name": "padim",
        "backbone": "resnet18",
        "pre_trained": True,
        "layers": ["layer1", "layer2", "layer3"],
        "normalization_method": "min_max",
        "input_size": [256, 256],
    }
}
model = get_model(model_config)
print(f"Model type: {type(model)}")

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: DeprecationWarning: From v0.10 an `'Binary*'`, `'Multiclass*', `'Multilabel*'` version now exist of each classification metric. Moving forward we recommend using these versions. This base metric will still work as it did prior to v0.10 until v0.11. From v0.11 the `task` argument introduced in this metric will be required and the general order of arguments may change, such that this metric will just function as an single entrypoint to calling the three specialized versions.
  warnings.warn(*args, **kwargs)
/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Model type: <class 'anomalib.models.padim.lightning_model.PadimLightning'>


## Training

Users can use each framework's training provided by OTX. (Engine)

- The engine requires the necessary models and DataLoaders for each framework.

In [6]:
work_path = "/tmp/OTX-API-test"
engine = Engine(work_dir=work_path)

engine.train(
    model=model,
    train_dataloader=train_dataloader,
    max_epochs=5,
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=2)`.
  rank_zero_warn(
/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/pytorch_lightning/core/optimizer.py:183: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(

  | Name                  | Type                     | Params
----------------------------------

Epoch 0:  32%|███▏      | 9/28 [00:00<00:00, 56.72it/s, loss=nan, v_num=4]

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:138: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Epoch 0:  43%|████▎     | 12/28 [00:00<00:00, 56.60it/s, loss=nan, v_num=4]

Epoch 4: 100%|██████████| 28/28 [00:00<00:00, 53.75it/s, loss=nan, v_num=4]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 28/28 [00:00<00:00, 40.34it/s, loss=nan, v_num=4]


{'model': PadimLightning(
   (image_threshold): AnomalyScoreThreshold()
   (pixel_threshold): AnomalyScoreThreshold()
   (model): PadimModel(
     (feature_extractor): FeatureExtractor(
       (feature_extractor): FeatureListNet(
         (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act1): ReLU(inplace=True)
         (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
         (layer1): Sequential(
           (0): BasicBlock(
             (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
             (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (drop_block): Identity()
             (act1): ReLU(inplace=True)
             (aa): Identity()
             (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(

In [7]:
engine.train(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    val_interval=1,
    max_epochs=2,
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | image_threshold       | AnomalyScoreThreshold    | 0     
1 | pixel_threshold       | AnomalyScoreThreshold    | 0     
2 | model                 | PadimModel               | 2.8 M 
3 | normalization_metrics | MinMax                   | 0     
4 | image_metrics         | AnomalibMetricCollection | 0     
5 | pixel_metrics         | AnomalibMetricCollection | 0     
-------------------------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.131    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1: 100%|██████████| 672/672 [01:29<00:00,  7.54it/s, loss=nan, v_num=5]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 672/672 [01:29<00:00,  7.53it/s, loss=nan, v_num=5]


{'model': PadimLightning(
   (image_threshold): AnomalyScoreThreshold()
   (pixel_threshold): AnomalyScoreThreshold()
   (model): PadimModel(
     (feature_extractor): FeatureExtractor(
       (feature_extractor): FeatureListNet(
         (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act1): ReLU(inplace=True)
         (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
         (layer1): Sequential(
           (0): BasicBlock(
             (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
             (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (drop_block): Identity()
             (act1): ReLU(inplace=True)
             (aa): Identity()
             (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(

## OTX AutoEngine (Automation Training API)
OTX provides a more convenient API called AutoEngine.

- It's more convenient for users to use Engine, which provides auto-configuration and the features provided by OTX without having to choose a framework.
- Prepare Dataset & DataLoader + Prepare Model + OTX Recipes + Training + ETC.
- This will make all of the above steps happen automatically. (Auto: Model Selection & build, Dataset Configuration, Training, etc..)

In [8]:
from otx.v2.api.core.engine import AutoEngine

output_dir = "/tmp/OTX-API-test"
data_roots = "../../../../tests/assets/anomaly/hazelnut/train"
default_config_path = "../configs/anomaly_classification/otx_anomalib_padim.yaml"

engine = AutoEngine(
    task="anomaly_classification",  # TODO: Need to add Automation-task detection for Anomaly
    work_dir=output_dir,
    train_data_roots=data_roots,
    config=default_config_path,
)

engine.train(batch_size=2, max_epochs=5)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_predict_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger f

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `ROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | image_threshold       | AnomalyScoreThreshold    | 0     
1 | pixel_threshold       | AnomalyScoreThreshold    | 0     
2 | model                 | PadimModel               | 2.8 M 
3 | normalization_metrics | MinMax                   | 0     
4 | image_metrics         | AnomalibMetricCollection | 0     
5 | pixel_metrics         | AnomalibMetricCollection | 0     
-------------------------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.131    Total estimated

Epoch 4: 100%|██████████| 14/14 [00:00<00:00, 62.19it/s, loss=nan, v_num=0]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 14/14 [00:00<00:00, 31.18it/s, loss=nan, v_num=0]


{'model': PadimLightning(
   (image_threshold): AnomalyScoreThreshold()
   (pixel_threshold): AnomalyScoreThreshold()
   (model): PadimModel(
     (feature_extractor): FeatureExtractor(
       (feature_extractor): FeatureListNet(
         (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act1): ReLU(inplace=True)
         (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
         (layer1): Sequential(
           (0): BasicBlock(
             (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
             (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (drop_block): Identity()
             (act1): ReLU(inplace=True)
             (aa): Identity()
             (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(